In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
from argparse import ArgumentParser

import torch
from voltaml.compile import VoltaGPUCompiler
from voltaml.inference import gpu_performance
import torchvision
import segmentation_models_pytorch as smp

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Model 

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


## Set parameters for FP16

In [3]:
input_shape = (1,3,224,224)
precision = 'fp16'
compiled_model_dir = 'deeplabv3_mobile.engine' ## Set Model dir
throughput_batch_size = 1

### Compile Model

In [4]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    simplify=True
)

compiled_model = compiler.compile()

-------- Loading ONNX ---------------


INFO:EngineBuilder:Network Description
Network Description
Network Description
INFO:EngineBuilder:Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Building fp16 Engine in /workspace/voltaML/demo/deeplabv3_mobile.engine
Building fp16 Engine in /workspace/voltaML/demo/deeplabv3_mobile.engine
Building fp16 Engine in /workspace/voltaML/demo/deeplabv3_mobile.engine


[08/23/2022-14:28:00] [TRT] [I] [MemUsageChange] Init CUDA: CPU +313, GPU +0, now: CPU 1111, GPU 682 (MiB)
[08/23/2022-14:28:00] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 1111 MiB, GPU 682 MiB
[08/23/2022-14:28:01] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 1246 MiB, GPU 716 MiB
[08/23/2022-14:28:01] [TRT] [W] parsers/onnx/onnx2trt_utils.cpp:364: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[08/23/2022-14:28:01] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +513, GPU +224, now: CPU 1802, GPU 940 (MiB)
[08/23/2022-14:28:01] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +116, GPU +52, now: CPU 1918, GPU 992 (MiB)
[08/23/2022-14:28:01] [TRT] [I] Local timing cache in use. Profiling results in this builder pass will not be stored.


INFO:EngineBuilder:Serializing engine to file: /workspace/voltaML/demo/deeplabv3_mobile.engine
Serializing engine to file: /workspace/voltaML/demo/deeplabv3_mobile.engine
Serializing engine to file: /workspace/voltaML/demo/deeplabv3_mobile.engine


[08/23/2022-14:30:05] [TRT] [I] Detected 1 inputs and 2 output network tensors.
[08/23/2022-14:30:05] [TRT] [I] Total Host Persistent Memory: 131088
[08/23/2022-14:30:05] [TRT] [I] Total Device Persistent Memory: 20725760
[08/23/2022-14:30:05] [TRT] [I] Total Scratch Memory: 0
[08/23/2022-14:30:05] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 23 MiB, GPU 735 MiB
[08/23/2022-14:30:05] [TRT] [I] [BlockAssignment] Algorithm ShiftNTopDown took 11.0196ms to assign 6 blocks to 128 nodes requiring 3462657 bytes.
[08/23/2022-14:30:05] [TRT] [I] Total Activation Memory: 3462657
[08/23/2022-14:30:05] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 2469, GPU 1236 (MiB)
[08/23/2022-14:30:05] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 2469, GPU 1244 (MiB)
[08/23/2022-14:30:05] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in building engine: CPU +21, GPU +22, now: CPU 21, GPU 22 (MiB)


In [5]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size, is_yolo=False)

calculating throughput: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 173.32it/s]


Latency:
--------------------------------------------------
VoltaML GPU Inference Latency: 1.58 ms / sample
PyTorch Inference Latency: 5.80 ms / sample


Throughput:
--------------------------------------------------
VoltaML GPU Inference Throughput: 659.81 samples / s
PyTorch Inference Throughput: 174.35 samples / s


### Set parameters for INT8

In [3]:
input_shape = (1,3,224,224)
precision = 'int8'
compiled_model_dir = 'deeplabv3_mobile_int8.engine' ## Compiled model directory
throughput_batch_size = 1
calib_input = '/workspace/val2017/' ## Calib input images path
calib_cache = '' ## Cache name
calib_num_images=25000
calib_batch_size=8
calib_preprocessor='V1MS'

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    calib_input=calib_input,
    calib_cache=calib_cache,
    calib_num_images=calib_num_images,
    calib_batch_size=calib_batch_size,
    calib_preprocessor=calib_preprocessor
)

compiled_model = compiler.compile()

INFO:EngineBuilder:Network Description
Network Description
Network Description
INFO:EngineBuilder:Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '619' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '638' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Building int8 Engine in /workspace/voltaML/demo/deeplabv3_mobile_int8.engine
Building int8 Engine in /workspace/voltaML/demo/deeplabv3_mobile_int8.engine
Building int8 Engine in /workspace/voltaML/demo/deeplabv3_mobile_in

[08/23/2022-14:40:15] [TRT] [I] [MemUsageChange] Init CUDA: CPU +313, GPU +0, now: CPU 1062, GPU 687 (MiB)
[08/23/2022-14:40:15] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 1062 MiB, GPU 687 MiB
[08/23/2022-14:40:16] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 1197 MiB, GPU 721 MiB
[08/23/2022-14:40:16] [TRT] [W] parsers/onnx/onnx2trt_utils.cpp:364: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[08/23/2022-14:40:16] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +514, GPU +224, now: CPU 1754, GPU 951 (MiB)
[08/23/2022-14:40:16] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +115, GPU +52, now: CPU 1869, GPU 1003 (MiB)
[08/23/2022-14:40:16] [TRT] [I] Timing cache disabled. Turning it on will improve builder speed.


INFO:EngineBuilder:Calibrating image 8 / 5000
Calibrating image 8 / 5000
Calibrating image 8 / 5000


[08/23/2022-14:40:17] [TRT] [I] Detected 1 inputs and 2 output network tensors.
[08/23/2022-14:40:17] [TRT] [I] Total Host Persistent Memory: 16000
[08/23/2022-14:40:17] [TRT] [I] Total Device Persistent Memory: 0
[08/23/2022-14:40:17] [TRT] [I] Total Scratch Memory: 0
[08/23/2022-14:40:17] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 0 MiB, GPU 43 MiB
[08/23/2022-14:40:17] [TRT] [I] [BlockAssignment] Algorithm ShiftNTopDown took 21.8467ms to assign 7 blocks to 176 nodes requiring 7831552 bytes.
[08/23/2022-14:40:17] [TRT] [I] Total Activation Memory: 7831552
[08/23/2022-14:40:18] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +1, GPU +8, now: CPU 2390, GPU 1273 (MiB)
[08/23/2022-14:40:18] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 2390, GPU 1281 (MiB)
[08/23/2022-14:40:18] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +10, now: CPU 2389, GPU 1257 (MiB)
[08/23/2022-14:40:18] [TRT] [I] [MemUsageChange] Init c

INFO:EngineBuilder:Calibrating image 16 / 5000
Calibrating image 16 / 5000
Calibrating image 16 / 5000
INFO:EngineBuilder:Calibrating image 24 / 5000
Calibrating image 24 / 5000
Calibrating image 24 / 5000
INFO:EngineBuilder:Calibrating image 32 / 5000
Calibrating image 32 / 5000
Calibrating image 32 / 5000
INFO:EngineBuilder:Calibrating image 40 / 5000
Calibrating image 40 / 5000
Calibrating image 40 / 5000


...

[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 613 in 0.0214298 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 614 in 0.0215062 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 615 in 0.02136 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 616 in 0.0214829 seconds.


INFO:EngineBuilder:Calibrating image 4944 / 5000
Calibrating image 4944 / 5000
Calibrating image 4944 / 5000
INFO:EngineBuilder:Calibrating image 4952 / 5000
Calibrating image 4952 / 5000
Calibrating image 4952 / 5000
INFO:EngineBuilder:Calibrating image 4960 / 5000
Calibrating image 4960 / 5000
Calibrating image 4960 / 5000
INFO:EngineBuilder:Calibrating image 4968 / 5000
Calibrating image 4968 / 5000
Calibrating image 4968 / 5000


[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 617 in 0.0215076 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 618 in 0.0215279 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 619 in 0.0215497 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 620 in 0.0222791 seconds.


INFO:EngineBuilder:Calibrating image 4976 / 5000
Calibrating image 4976 / 5000
Calibrating image 4976 / 5000
INFO:EngineBuilder:Calibrating image 4984 / 5000
Calibrating image 4984 / 5000
Calibrating image 4984 / 5000
INFO:EngineBuilder:Calibrating image 4992 / 5000
Calibrating image 4992 / 5000
Calibrating image 4992 / 5000
INFO:EngineBuilder:Calibrating image 5000 / 5000
Calibrating image 5000 / 5000
Calibrating image 5000 / 5000
INFO:EngineBuilder:Finished calibration batches
Finished calibration batches
Finished calibration batches


[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 621 in 0.0212666 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 622 in 0.0215321 seconds.
[08/23/2022-14:40:58] [TRT] [I]   Calibrated batch 623 in 0.0220368 seconds.
[08/23/2022-14:40:59] [TRT] [I]   Calibrated batch 624 in 0.0213648 seconds.


In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size, is_yolo=False)